# Green (function) expansion and 2-photon decay rates

In [1]:
using JAC

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-9421716996860947222\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-14142809031172620157\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-9421716996860947222\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Green functions are primarely used in mathematical physics to represent solutions of inhomogeneous boundary-value problems as they frequently occur, for instance, in scattering theory. In JAC, approximate (many-electron) Green functions are typically applied in order to sum over some -- more or less complete -- spectrum and, hence, to compute second- and higher-order processes. In JAC, moreover, a Green function representation refers to just one or a few multiplets of different (level) symmetries $J_1,\: J_2,\: ....$, i.e. lists of  many-electron levels with up to $n^{(max)}$ electrons within the continuum. 

In JAC, indeed, each Green function is classified by the (maximum) number of free electrons and a pre-specified (de-) excitation scheme that is to be applied in order to generate from a list of given reference configurations all those (nonrelativistic) configurations that are to be considered in the many-electron (CSF) basis of the Green function representation. To understand such a representation in more detail, let us first consider the internal set-up of a general `Representation` and a `GreenExpansion` as a particular representation:

In [2]:
? Atomic.Representation

`struct  Atomic.Representation`       ... a struct for defining an atomic representation. Such representations often refer to approximate wave function approximations of         one or several levels but may concern also a mean-field basis (for some multiplet of some given configurations) or Green functions,         etc.

```
+ name             ::String                      ... to assign a name to the given model.
+ nuclearModel     ::Nuclear.Model               ... Model, charge and parameters of the nucleus.
+ grid             ::Radial.Grid                 ... The radial grid to be used for the computation.
+ refConfigs       ::Array{Configuration,1}      ... List of references configurations, at least 1.
+ repType          ::AbstractRepresentationType  ... Specifies the particular representation.
```

---

`Atomic.Representation()`  ... constructor for an 'empty' instance of the a variable::Atomic.Representation


In [3]:
? Atomic.GreenExpansion

`struct  Atomic.GreenExpansion  <:  AbstractRepresentationType`       ... defines a type to keep an (approximate) Green (function) expansion that is associated with a given set of reference         configurations.

```
+ approach          ::Atomic.AbstractGreenApproach    ... Approach used to approximate the representation.
+ excitationScheme  ::Basics.AbstractExcitationScheme ... Applied excitation scheme w.r.t. refConfigs. 
+ levelSymmetries   ::Array{LevelSymmetry,1}          ... Total symmetries J^P to be included into Green expansion.
+ NoElectrons       ::Int64                           ... Number of electrons.
+ settings          ::Atomic.GreenSettings            ... settings for the Green (function) expansion.
```

---

`Atomic.GreenExpansion()`  ... constructor for an `empty` instance of Atomic.GreenExpansion.


Like all other representations, a Green (function) expansion is first of all based on a nuclear model, a set of reference configurations as well as a pre-defined grid. The `GreenExpansion` itself can be generated for different excitation schemes (with regard to the reference configurations) and for a given list of level symmetries. The `AbstractGreenApproach` determines how the electron-electron interaction is treated in the set-up and diagonalization of the `GreenChannel`'s, i.e. the corresponding Hamiltonian matrices:

In [4]:
? Atomic.AbstractGreenApproach

`abstract type Atomic.AbstractGreenApproach`      ... defines an abstract and a number of singleton types for approximating a many-electron Green         function expansion for calculating second-order processes.

  * struct SingleCSFwithoutCI         ... to approximate the many-electron multiplets (gMultiplet) for every chosen level symmetry by dealing with each CSF      independently, and without any configuration interaction. This is a fast but also very rough approximation.
  * struct CoreSpaceCI                 ... to approximate the many-electron multiplets (gMultiplet) by taking the electron-electron interaction between the      bound-state orbitals into account.
  * struct DampedSpaceCI                 ... to approximate the many-electron multiplets (gMultiplet) by taking the electron-electron interaction for all, the      bound and free-electron, orbitals into account but by including a damping factor e^{tau*r}, tau > 0 into the      electron densities rho*ab (r) –> rho*ab (r) * e^{tau*r}


What is seen ... and which information needed to be provided for such a computation

In [5]:
? GreenChannel

search: GreenChannel



`struct  Atomic.GreenChannel`       ... defines a type for a single symmetry channel of an (approximate) Green (function) expansion.

```
+ symmetry          ::LevelSymmetry    ... Level symmetry of this part of the representation.
+ gMultiplet        ::Multiplet        ... Multiplet of (scattering) levels of this symmetry.
```

---

`Atomic.GreenChannel()`  ... constructor for an `empty` instance of Atomic.GreenChannel.


The *outcome* of a Green (function) expansion or representation is a list of `GreenChannel`'s which can be obtained as usual from some dictionary. Apart from the `AbstractGreenApproach` and the `AbstractExcitationScheme`, further details about the expansion can be specified and controlled by means of the `GreenSettings`:

In [6]:
? GreenSettings

search: GreenSettings



`struct  Atomic.GreenSettings`       ... defines a type for defining the details and parameters of the approximate Green (function) expansion.

```
+ nMax                     ::Int64            ... maximum principal quantum numbers of (single-electron) 
                                                  excitations that are to be included into the representation.
+ lValues                  ::Array{Int64,1}   ... List of (non-relativistic) orbital angular momenta for which
                                                  (single-electron) excitations are to be included.
+ printBeforeComputation   ::Bool             ... True if a short overview is to be printed before. 
+ selectLevels             ::Bool             ... True if individual levels are selected for the computation.
+ selectedLevels           ::Array{Int64,1}   ... List of selected levels.
```

---

`Atomic.GreenSettings()`  ... constructor for an `empty` instance of Atomic.GreenSettings.


In these settings, one can specify the maximum principle quantum number and the orbital angular momenta for the nonrelativistic $(n,\ell)$ shells which are considered for virtual excitations. These orbitals correspond to those orbitals to which electrons can be excited, for instance, in a RAS expansion.

In this tutorial, we wish to generate a Green (function) expansion that is suitable to estimate the E1-M1 rates for the two-photon decay of the $1s^2 2s 2p\; ^3P_0$ lowest-excited level of beryllium-like ions. This level is known as one of the longest-living levels of atomic ions with lifetimes up to millions of years near to the neutral end. We shall consider the E1-M1 two-photon decay of beryllium-like xenon with an expected lifetime of about ... second. 

Indeed, the $1s^2 2s 2p\; ^3P_0 - 1s^2 2s^2\; ^1S_0$ decay is (highly) forbidden with regard to any single-photon decay as well as to E1E1 two-photon transition. For an isolated atom, the lowest-order non-vanishing decay refers to a E1M1 decay. To compute the E1M1 rates, we need Green function channels of symmetry $J^P = 1+$ (for the E1M1 transition amplitudes) and $1-$ (for the M1E1 transition amplitudes). In JAC, we can generate these channels by:

In [7]:
name          = "Beryllium 1s^2 2s np J=1- and  1s^2 2s ns J=1- Green channels"
refConfigs    = [Configuration("[He] 2s^2"), Configuration("[He] 2s 2p")]
greenSettings = GreenSettings(5, [0, 1, 2], true, false, Int64[])

nMax:                     5  
lValues:                  [0, 1, 2]  
printBeforeComputation:   true  
selectLevels:             false  
selectedLevels:           Int64[]  


In [8]:
wa          = Representation(name, Nuclear.Model(54.), Radial.Grid("grid: exponential"), refConfigs, 
                             GreenExpansion( Atomic.SingleCSFwithoutCI(), Basics.DeExciteSingleElectron(), 
                                             [LevelSymmetry(1, Basics.plus), LevelSymmetry(1, Basics.minus)], 4, greenSettings) )

Define a radial grid of type MeshGL with 400 grid points
 [rnt=2.000e-06, h=5.000e-02, hp=0.000e+00, NoPoints=390, r_max=9.161e+02;
  B-splines with break points at every 7th point, nsL=56, nsS=57, orderL=7, orderS=8, orderGL=7] 


Atomic representation:   Beryllium 1s^2 2s np J=1- and  1s^2 2s ns J=1- Green channels for Z = 54.0 and with reference configurations: 
   1s^2 2s^2 ,  1s^2 2s^1 2p^1 ,  
representation type:   Green (function) expansion in JAC.Atomic.SingleCSFwithoutCI() approach and for excitation scheme  JAC.Basics.DeExciteSingleElectron(),
including (Green function channels with) symmetries LevelSymmetry[1 +, 1 -]:
... and the current settings:
nMax:                     5  
lValues:                  [0, 1, 2]  
printBeforeComputation:   true  
selectLevels:             false  
selectedLevels:           Int64[]  
  
  
nuclearModel:          Fermi nuclear model for Z = 54.0 with mass = 122.58, radius R = 4.722849091608491 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0.  
grid:                  Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = MeshGL, ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...

In [9]:
wb = generate(wa, output=true)


... in perform['computation: SCF'] ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^0 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^1 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 54.0 with mass = 122.58, radius R = 4.722849091608491 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.51935928e+03    -1.51947334e+03    +7.50765232e-05    
      2      2s_1/2      -3.83773854e+02    -3.83790162e+02    +4.24935903e-05    
     

Dict{String,Any} with 1 entry:
  "Green channels" => GreenChannel[symmetry:                1 +  …

In [10]:
As usual, we can access the (list of) Green function channels by:

LoadError: syntax: extra token "usual" after end of expression

In [11]:
wc = wb["Green channels"]

2-element Array{GreenChannel,1}:
 symmetry:                1 +  
gMultiplet:              name:        SingleCSFwithoutCI multiplet for 1 +  
levels:      Level[Level: J = 1, M = 1, parity = plus, index = 0 
energy:         -2366.4992644957674  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  
, Level: J = 1, M = 1, parity = plus, index = 0 
energy:         -2354.6289499145505  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

We will use these channels to *estimate* the two-photon E1M1 decay rates. ...

# **This nootebook is still under construction !!!**